In [2]:
import pymongo
import pandas as pd
import numpy as np
import math
from datetime import datetime

import dask.dataframe as dd ## 대규모 데이터 처리
# import vaex ## 대규모 데이터 처리

# from sklearn.linear_model import LinearRegression ## likes, dislikes 대체
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

import os
from dotenv import load_dotenv

In [3]:
file_path = 'C:/py_src/awake/data/'

In [4]:
# 계정 / 콘텐츠 분석 데이터셋 불러오기
merge_df_users_fin = pd.read_csv(file_path + 'merge_df_users_final3.csv', low_memory=False)
youtube_videos = pd.read_csv(file_path + 'youtube_videos.csv')

## 계정 데이터 분석

### 데이터 분할
- 미래를 예측하기 위한 모델이므로 시간순으로 데이터 분할
- 시작날짜 : 2023-03-26
- 종료날짜 : 2024-05-06
- 전체 기간의 80% 날짜 : 2024-02-11

In [7]:
# 컬럼 정리
unique_col = merge_df_users_fin.columns[:11]
x_col = merge_df_users_fin.columns[11:].drop('estimated_ad_revenue') ## y값 제거

In [8]:
# 데이터 분할
train_data = merge_df_users_fin[merge_df_users_fin['date'] <= '2024-02-11']
test_data = merge_df_users_fin[merge_df_users_fin['date'] > '2024-02-11']

In [9]:
# 불균형 확인
print(train_data.shape)
print(test_data.shape)

(84473, 57)
(21210, 57)


### 모델 기법 적용

In [10]:
# 각 모델 정의
rf_model = RandomForestRegressor(random_state=42)
gbm_model = GradientBoostingRegressor(random_state=42)
lgbm_model = LGBMRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(random_state=42)

# 모델 리스트
models = {
    'RandomForest': rf_model,
    'GBM': gbm_model,
    'LightGBM': lgbm_model,
    'XGBoost': xgb_model
}

In [9]:
# # 모델별 교차 검증 결과 저장
# results = {}

# for model_name, model in models.items():
#     print(f"\n{model_name} 모델 성능 평가 중...")
    
#     # 교차 검증
#     cv_scores = cross_val_score(model, train_data[x_col], train_data['subscribers_count'], cv=5, scoring='neg_mean_squared_error')
    
#     # 평균 RMSE 계산
#     rmse_scores = np.sqrt(-cv_scores)  # neg_mean_squared_error는 음수이므로 양수로 변환 후 제곱근
#     mean_rmse = rmse_scores.mean()
#     print(f"{model_name} 교차 검증 평균 RMSE: {mean_rmse}")
    
#     # 결과 저장
#     results[model_name] = mean_rmse

### 모델 성능 평가

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

for model in models:
    # 모델 정의 및 학습
    models[model].fit(train_data[x_col], train_data['estimated_ad_revenue'])

    # 예측
    y_pred = models[model].predict(test_data[x_col])

    # MSE 계산
    mse = mean_squared_error(test_data['estimated_ad_revenue'], y_pred)

    # RMSE 계산
    rmse = np.sqrt(mse)

    # R² 값 계산
    r2 = r2_score(test_data['estimated_ad_revenue'], y_pred)

    # Adjusted R² 계산
    n = len(test_data)  # 샘플 수
    p = test_data.shape[1]  # 독립 변수(특성) 수
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

    # 결과 출력
    print(model)
    print(f"R² 값: {r2:.4f}")
    print(f"Adjusted R² 값: {adjusted_r2:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print("----------------------------------------")
    print("")

RandomForest
R² 값: 0.9936
Adjusted R² 값: 0.9936
MSE: 47091405.1252
RMSE: 6862.3178
----------------------------------------

GBM
R² 값: 0.9974
Adjusted R² 값: 0.9974
MSE: 18992570.5474
RMSE: 4358.0466
----------------------------------------

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008939 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11454
[LightGBM] [Info] Number of data points in the train set: 84473, number of used features: 45
[LightGBM] [Info] Start training from score 14667.681766
LightGBM
R² 값: 0.9223
Adjusted R² 값: 0.9221
MSE: 571114131.6523
RMSE: 23897.9943
----------------------------------------

XGBoost
R² 값: 0.9663
Adjusted R² 값: 0.9662
MSE: 248007649.8104
RMSE: 15748.2586
----------------------------------------



In [92]:
# # 비선형모델 활용 변수 선정
# importances_df = pd.DataFrame({
#     'features': x_col,
#     'rf_importance': models['RandomForest'].feature_importances_,
#     'gbm_importance': models['GBM'].feature_importances_,
#     'lgbm_importance': models['LightGBM'].feature_importances_,
#     'xgb_importance': models['XGBoost'].feature_importances_    
# })
# importances_df['lgbm_importance'] = importances_df['lgbm_importance'] / importances_df['lgbm_importance'].sum() ## 다른 모델과 중요도 단위 다르기 때문에 정규화
# importances_df['mean_importance'] = importances_df[['rf_importance', 'gbm_importance', 'lgbm_importance', 'xgb_importance']].mean(axis=1) ## 각 변수별 모델 중요도 평균

### 모델 성능 개선

스태킹

In [14]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# 스태킹 모델 정의
stacking_model = StackingRegressor(
    estimators=[
    ('random_forest', models['RandomForest']),
    ('gbm', models['GBM']),
    ('lightgbm', models['LightGBM']),
    ('xgboost', models['XGBoost'])
    ],
    final_estimator=Ridge()
)

# 스태킹 모델 학습
stacking_model.fit(train_data[x_col], train_data['estimated_ad_revenue'])

# 예측
y_pred = stacking_model.predict(test_data[x_col])

# 성능 평가
# R² 값 계산
r2 = r2_score(test_data['estimated_ad_revenue'], y_pred)

# Adjusted R² 계산
n = len(test_data)  # 샘플 수
p = test_data.shape[1]  # 독립 변수(특성) 수
adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

# MSE 계산
mse = mean_squared_error(test_data['estimated_ad_revenue'], y_pred)

# RMSE 계산
rmse = mean_squared_error(test_data['estimated_ad_revenue'], y_pred, squared=False)

print(f"스태킹 앙상블 모델 R2: {r2:.4f}")
print(f"스태킹 앙상블 모델 Adjusted R2: {adjusted_r2:.4f}")
print(f"스태킹 앙상블 모델 MSE: {mse:.4f}")
print(f"스태킹 앙상블 모델 RMSE: {rmse:.4f}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006312 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11454
[LightGBM] [Info] Number of data points in the train set: 84473, number of used features: 45
[LightGBM] [Info] Start training from score 14667.681766
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11453
[LightGBM] [Info] Number of data points in the train set: 67578, number of used features: 45
[LightGBM] [Info] Start training from score 17638.612776
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006035 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11455
[LightGBM] [Info] Number of data points in the train set: 67578, number of used features: 45
[LightGBM] [Info

c:\Users\kkims\anaconda3\envs\awake\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 실제데이터 결과 확인

In [16]:
# 예측 결과 확인
y_pred = stacking_model.predict(merge_df_users_fin[x_col])
merge_df_users_fin['predict'] = y_pred

In [61]:
# 계정별 구독자수 평균, 구독자 예측수 평균 비교
result_df = merge_df_users_fin[['youtube_user_id','date','channel_title','estimated_ad_revenue','predict']]

# 1개월, 3개월, 6개월, 1년 뒤 구독자수 예측값
result_df['date'] = pd.to_datetime(result_df['date']) 

result_df['1_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-30).rolling(window=30).sum()
result_df['3_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-90).rolling(window=90).sum()
result_df['6_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-180).rolling(window=180).sum()
result_df['12_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-365).rolling(window=365).sum()

# 계정별 기간에 따른 광고 수익 예측값 비교
# result_df_final = result_df.groupby(['youtube_user_id'])[['estimated_ad_revenue','predict','1_month_future_predict','3_month_future_predict','6_month_future_predict','12_month_future_predict']].sum().reset_index()

# 계정별로 최종 평균값을 계산
result_df_final = result_df.groupby('youtube_user_id').agg({
    'estimated_ad_revenue': 'mean', ## 1일 평균 광고수익
    'predict': 'mean', ## 1일 평균 광고수익 예측
    '1_month_future_predict': 'mean',
    '3_month_future_predict': 'mean',
    '6_month_future_predict': 'mean',
    '12_month_future_predict': 'mean'
}).reset_index()

C:\Users\kkims\AppData\Local\Temp\ipykernel_6376\3275880359.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['date'] = pd.to_datetime(result_df['date'])
C:\Users\kkims\AppData\Local\Temp\ipykernel_6376\3275880359.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['1_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-30).rolling(window=30).sum()
C:\Users\kkims\AppData\Local\Temp\ipykernel_6376\3275880359.py:8: SettingWithCopyWarning: 
A value is trying to 

In [162]:
result_df[result_df['youtube_user_id'] == '63d2239450eb530dfd137d1e']

,youtube_user_id,date,channel_title,estimated_ad_revenue,predict,1_month_future_predict,3_month_future_predict,6_month_future_predict,12_month_future_predict
43071,63d2239450eb530dfd137d1e,2023-03-26,이고 EGO,297.806,44.758478,NaN,NaN,NaN,NaN
43072,63d2239450eb530dfd137d1e,2023-03-27,이고 EGO,252.163,-71.941941,NaN,NaN,NaN,NaN
43073,63d2239450eb530dfd137d1e,2023-03-28,이고 EGO,236.273,12.215801,NaN,NaN,NaN,NaN
43074,63d2239450eb530dfd137d1e,2023-03-29,이고 EGO,269.039,49.785959,NaN,NaN,NaN,NaN
43075,63d2239450eb530dfd137d1e,2023-03-30,이고 EGO,503.468,128.037987,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
43471,63d2239450eb530dfd137d1e,2024-04-29,이고 EGO,104.513,-133.691835,NaN,NaN,NaN,NaN
43472,63d2239450eb530dfd137d1e,2024-04-30,이고 EGO,80.377,-143.835546,NaN,NaN,NaN,NaN
43473,63d2239450eb530dfd137d1e,2024-05-01,이고 EGO,28.881,-160.208379,NaN,NaN,NaN,NaN
43474,63d2239450eb530dfd137d1e,2024-05-02,이고 EGO,53.548,-153.921827,NaN,NaN,NaN,NaN


In [148]:
result_df_final[result_df_final['predict'] > 0].sort_values('predict')

,youtube_user_id,estimated_ad_revenue,predict,1_month_future_predict,3_month_future_predict,6_month_future_predict,12_month_future_predict
48,62b718bc507271632b8f0ce4,339.897065,15.990677,4.692295e+02,-2.621279e+02,-3.614523e+03,NaN
201,64bce3c5616bd20e3037a1cf,323.219489,23.331282,-1.257770e+02,-6.636114e+03,-1.884004e+04,NaN
98,63d2239450eb530dfd137d1e,291.510282,37.633269,8.248158e+02,1.640833e+03,-5.892702e+03,NaN
76,63315973ef33d840a0999698,367.495198,47.274911,2.638626e+03,1.863610e+04,3.914999e+04,NaN
151,6417c62789085e280d0e410b,350.699542,47.398836,-8.481259e+02,-1.499391e+04,-3.482298e+04,NaN
...,...,...,...,...,...,...,...
142,640339ac118c0f5858818694,157229.117648,159944.491440,4.721605e+06,1.344499e+07,2.493102e+07,NaN
57,62d11f080b4c4c7502a5be3d,326569.531929,332969.983834,9.928727e+06,3.084473e+07,6.230265e+07,NaN
84,639bb8dcd603b8138e33780b,342046.202626,348007.035268,1.123271e+07,3.696964e+07,1.215998e+08,NaN
108,63eb4f87ee122e631992279f,343644.047387,350449.210011,9.967546e+06,3.010190e+07,5.184346e+07,NaN


In [65]:
result_df[result_df['youtube_user_id'].isin(result_df_final[result_df_final['predict'] < 0]['youtube_user_id'])]['channel_title'].unique()

array(['고도람 Go!doram', '0', '세남자 물고기', '름쿠 ᴘʟᴀʏʟɪꜱᴛ', '달고캠핑',
       '고군 Gohgoon', '루깬미', '임삐나', '바른걸음연구소', '임퓨의 비트메이킹 클래스',
       '혜성네일_comet', '마파TV', '담순언니 Twins Vlog', '유익한 균튜버', '1분뉴스',
       'the sence', 'MINLEE 민리', '성한준', '다먹어라이언',
       '하부유튜브 Minor / (Lower) YouTube', 'sa lly', '수집의 수집', '도아이 Doh-I',
       'OBL - 온라인 농부, 사자가 되다', '서유 SEOYU DANCE', 'ORlGN 오리진', '윈플즈TV',
       'DDONIE 또니 / 러브크레센트', '슈로시안 SUROSIAN', '김밈서', '드론브이로그 DroneVlog',
       'gahyun 가현', 'Mein 미인', '김두부', '은는이가', '인썸니아TV', '한나임한나Hannaim',
       '수란쿤', '기자 황덕현 KIJA HWANG', '콜로니', '어웨이커 | 크리에이터 이코노미',
       'GMENCY 멘시의 마인크래프트', '로컬필름 LOCAL FILM', '꾸앤끄', '키키낙낙',
       '프롬수지 fromsuzy', 'fromsuzy 프롬수지', '탬니몰리', 'WORKS.D PLAYLIST',
       '강포동하우스', '태다린tae_darin', '홈바부부_HOMBA BOOBOO', 'TJ 영상채널',
       'MORE김모어', '청도시네마', '두꼽이Challenge', '여행윤Tripyun', '핸슥슥',
       '연디아 채널 Yeondia Channel', 'D_tail_디테', '김희영', '김우다', '닷츠 DOTS',
       '약사 이진수💊', '모리녀', '자수의숲jasooforest', 'SBM&E Official',
       'Yeren

In [115]:
# 상위, 하위 20개 계정
top_user_id_20_6_month = result_df_final[result_df_final['6_month_future_predict'] > 0].sort_values(['6_month_future_predict'],ascending=False)['youtube_user_id'].iloc[:20]
bottom_user_id_20_6_month = result_df_final[result_df_final['6_month_future_predict'] > 0].sort_values(['6_month_future_predict'],ascending=False)['youtube_user_id'].iloc[-20:]

In [112]:
print(len(result_df[result_df['youtube_user_id'].isin(top_user_id_20_6_month)]['channel_title'].unique()))
print(result_df[result_df['youtube_user_id'].isin(top_user_id_20_6_month)]['channel_title'].unique())

20
['abbapraise 아바프레이즈' '0' 'Jeffreyxking' '팀브라더스' 'kiu기우쌤'
 '수빙수tv sooBingsoo' '너굴몬' '미니멀영어 Minimal English' '日本ジヌ【니혼지누】ー韓国に関する全て'
 '뻘짓연구소' '정가거부' '북토크' '집구석구석꿀팁, 집꿀' '나연이즈백 LPGA Na Yeon Choi' '유네린NERIN'
 '뷰드름 유튜버 인씨' '빅민 GAME' 'OSSC' '이현우의 MLBTV' '하원장 강동현']


In [113]:
print(len(result_df[result_df['youtube_user_id'].isin(bottom_user_id_20_6_month)]['channel_title'].unique()))
print(result_df[result_df['youtube_user_id'].isin(bottom_user_id_20_6_month)]['channel_title'].unique())

20
['띠혜 ddihye' '0' '소리미의 신화방송' '나는 불독' 'assesta' '배우GO' '굥플레이스 맛집투어'
 '이숲soop' '원의 독백' '채림처럼firstcherry' '모하지연 MOHAJIYEON' '오토컨테이너 스튜디오'
 '부반TV_부에 반하다' '미디하는남자' '이고 EGO' '맛집남자 foodman' '-mentalholder 멘탈홀더 tv'
 '너드 슬로리 SloLee' '캠핑 릴리아빠' '여리여리YeoriYeori']


In [117]:
len(set(top_user_id_20_6_month) & set(top_user_id_20_6_month) & set(top_user_id_20_6_month))

16

## 콘텐츠 데이터 분석

In [5]:
# 최종 콘텐츠 분석 데이터셋
# 환율 적용 - 해당기간 평균환율 : 1322.42
exchange_rate_col = ['estimatedRevenue', 'estimatedAdRevenue', 'estimatedRedPartnerRevenue', 'grossRevenue', 'cpm', 'playbackBasedCpm']
youtube_videos[exchange_rate_col] = youtube_videos[exchange_rate_col] * 1322.42

In [6]:
# 잘못된값 처리
youtube_videos['estimatedRevenue'] = np.where(youtube_videos['estimatedRevenue'] < 0,
                                               youtube_videos['estimatedAdRevenue'] + youtube_videos['estimatedRedPartnerRevenue'],
                                               youtube_videos['estimatedRevenue'])

In [7]:
# 버그로 사용된 수치값 대체
youtube_videos['likes'] = np.where(youtube_videos['likes'] < 0, 0, youtube_videos['likes'])
youtube_videos['dislikes'] = np.where(youtube_videos['dislikes'] < 0, 0, youtube_videos['dislikes'])

### 중요지표확인

In [8]:
youtube_videos.columns

Index(['youtube_user_id', 'video', 'end_date', 'views', 'redViews', 'comments',
       'likes', 'dislikes', 'shares', 'estimatedMinutesWatched',
       'estimatedRedMinutesWatched', 'averageViewDuration',
       'averageViewPercentage', 'videosAddedToPlaylists',
       'videosRemovedFromPlaylists', 'estimatedRevenue', 'estimatedAdRevenue',
       'grossRevenue', 'estimatedRedPartnerRevenue', 'playbackBasedCpm', 'cpm',
       'subscribersGained', 'subscribersLost', 'monetizedPlaybacks',
       'adImpressions', 'cardClickRate', 'cardTeaserClickRate',
       'cardImpressions', 'cardTeaserImpressions', 'cardClicks',
       'cardTeaserClicks'],
      dtype='object')

In [9]:
# y값 설정
# youtube_videos['estimatedRevenue']
# youtube_videos['net_subscribers_change']
# youtube_videos['engage_rate']
youtube_videos['net_subscribers_change'] = youtube_videos['subscribersGained'] - youtube_videos['subscribersLost']
youtube_videos['engage_rate'] = (youtube_videos['likes'] + youtube_videos['comments'] + youtube_videos['shares']) / youtube_videos['views']

In [10]:
# 조회수당 수익
youtube_videos['revenue_per_view'] = youtube_videos['estimatedRevenue'] / youtube_videos['views']

# 구독자당 수익
youtube_videos['revenue_per_Subscriber'] = youtube_videos['estimatedRevenue'] / youtube_videos['subscribersGained']

# YouTube Premium 수익
youtube_videos['estimatedRedPartnerRevenue']

# 수익 다변화 비율
youtube_videos['revenue_diversification_ratio'] = (youtube_videos['grossRevenue'] - youtube_videos['estimatedRevenue']) / youtube_videos['estimatedRevenue']

# 구독자 증가율
youtube_videos['subscriber_growth_rate'] = youtube_videos['subscribersGained'] / (youtube_videos['subscribersGained'] + youtube_videos['subscribersLost'])

# 구독자 감소율
youtube_videos['subscriber_loss_rate'] = youtube_videos['subscribersLost'] / (youtube_videos['subscribersGained'] + youtube_videos['subscribersLost'])

# 구독자 유지율
youtube_videos['subscriber_retention_rate'] = (youtube_videos['subscribersGained'] - youtube_videos['subscribersLost']) / youtube_videos['subscribersGained']

# 콘텐츠당 구독자 증가율
youtube_videos['subscriber_gain_per_content'] = youtube_videos['subscribersGained']/ youtube_videos['videosAddedToPlaylists']

# 구독자당 시청 시간
youtube_videos['watch_time_per_subscriber'] = youtube_videos['estimatedMinutesWatched'] / youtube_videos['subscribersGained']

# 광고재생률
youtube_videos['ad_playback_rate'] = youtube_videos['monetizedPlaybacks'] / youtube_videos['adImpressions']

상관분석

In [27]:
corr_df = youtube_videos[youtube_videos.columns[3:]].corr()
corr_df[['estimatedRevenue','net_subscribers_change','engage_rate']]

,estimatedRevenue,net_subscribers_change,engage_rate
views,0.318394,0.697500,0.017541
redViews,0.412902,0.414674,0.011004
comments,0.104338,0.129862,0.069930
likes,0.244780,0.568465,0.017327
dislikes,0.267804,0.740121,0.015847
shares,0.268022,0.585073,0.022817
estimatedMinutesWatched,0.699625,0.560338,0.012668
estimatedRedMinutesWatched,0.682464,0.169521,0.002882
averageViewDuration,0.080220,-0.004228,0.008303
averageViewPercentage,-0.001652,0.008430,0.020841


### 기간별 피처 생성

In [11]:
# 주별 피처 생성(콘텐츠별로 계산)
youtube_videos['weekly_revenue_per_view_std'] = youtube_videos.groupby('video')['revenue_per_view'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_revenue_per_Subscriber_std'] = youtube_videos.groupby('video')['revenue_per_Subscriber'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_estimatedRedPartnerRevenue_std'] = youtube_videos.groupby('video')['estimatedRedPartnerRevenue'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_revenue_diversification_ratio_std'] = youtube_videos.groupby('video')['revenue_diversification_ratio'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_subscriber_growth_rate_std'] = youtube_videos.groupby('video')['subscriber_growth_rate'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_subscriber_loss_rate_std'] = youtube_videos.groupby('video')['subscriber_loss_rate'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_subscriber_retention_rate_std'] = youtube_videos.groupby('video')['subscriber_retention_rate'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_subscriber_gain_per_content_std'] = youtube_videos.groupby('video')['subscriber_gain_per_content'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_watch_time_per_subscriber_std'] = youtube_videos.groupby('video')['watch_time_per_subscriber'].transform(lambda x: x.rolling(window=7).std())
youtube_videos['weekly_ad_playback_rate_std'] = youtube_videos.groupby('video')['ad_playback_rate'].transform(lambda x: x.rolling(window=7).std())

youtube_videos['weekly_revenue_per_view_trd'] = youtube_videos.groupby('video')['revenue_per_view'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_revenue_per_Subscriber_trd'] = youtube_videos.groupby('video')['revenue_per_Subscriber'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_estimatedRedPartnerRevenue_trd'] = youtube_videos.groupby('video')['estimatedRedPartnerRevenue'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_revenue_diversification_ratio_trd'] = youtube_videos.groupby('video')['revenue_diversification_ratio'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_subscriber_growth_rate_trd'] = youtube_videos.groupby('video')['subscriber_growth_rate'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_subscriber_loss_rate_trd'] = youtube_videos.groupby('video')['subscriber_loss_rate'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_subscriber_retention_rate_trd'] = youtube_videos.groupby('video')['subscriber_retention_rate'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_subscriber_gain_per_content_trd'] = youtube_videos.groupby('video')['subscriber_gain_per_content'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_watch_time_per_subscriber_trd'] = youtube_videos.groupby('video')['watch_time_per_subscriber'].transform(lambda x: (x - x.shift(7)) / x.shift(7))
youtube_videos['weekly_ad_playback_rate_trd'] = youtube_videos.groupby('video')['ad_playback_rate'].transform(lambda x: (x - x.shift(7)) / x.shift(7))

# 월별 피처 생성(콘텐츠별로 계산)
youtube_videos['monthly_revenue_per_view_std'] = youtube_videos.groupby('video')['revenue_per_view'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_revenue_per_Subscriber_std'] = youtube_videos.groupby('video')['revenue_per_Subscriber'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_estimatedRedPartnerRevenue_std'] = youtube_videos.groupby('video')['estimatedRedPartnerRevenue'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_revenue_diversification_ratio_std'] = youtube_videos.groupby('video')['revenue_diversification_ratio'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_subscriber_growth_rate_std'] = youtube_videos.groupby('video')['subscriber_growth_rate'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_subscriber_loss_rate_std'] = youtube_videos.groupby('video')['subscriber_loss_rate'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_subscriber_retention_rate_std'] = youtube_videos.groupby('video')['subscriber_retention_rate'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_subscriber_gain_per_content_std'] = youtube_videos.groupby('video')['subscriber_gain_per_content'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_watch_time_per_subscriber_std'] = youtube_videos.groupby('video')['watch_time_per_subscriber'].transform(lambda x: x.rolling(window=30).std())
youtube_videos['monthly_ad_playback_rate_std'] = youtube_videos.groupby('video')['ad_playback_rate'].transform(lambda x: x.rolling(window=30).std())

youtube_videos['monthly_revenue_per_view_trd'] = youtube_videos.groupby('video')['revenue_per_view'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_revenue_per_Subscriber_trd'] = youtube_videos.groupby('video')['revenue_per_Subscriber'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_estimatedRedPartnerRevenue_trd'] = youtube_videos.groupby('video')['estimatedRedPartnerRevenue'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_revenue_diversification_ratio_trd'] = youtube_videos.groupby('video')['revenue_diversification_ratio'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_subscriber_growth_rate_trd'] = youtube_videos.groupby('video')['subscriber_growth_rate'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_subscriber_loss_rate_trd'] = youtube_videos.groupby('video')['subscriber_loss_rate'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_subscriber_retention_rate_trd'] = youtube_videos.groupby('video')['subscriber_retention_rate'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_subscriber_gain_per_content_trd'] = youtube_videos.groupby('video')['subscriber_gain_per_content'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_watch_time_per_subscriber_trd'] = youtube_videos.groupby('video')['watch_time_per_subscriber'].transform(lambda x: (x - x.shift(30)) / x.shift(30))
youtube_videos['monthly_ad_playback_rate_trd'] = youtube_videos.groupby('video')['ad_playback_rate'].transform(lambda x: (x - x.shift(30)) / x.shift(30))

In [12]:
# null값 대체
youtube_videos = youtube_videos.fillna(0) ## NaN
youtube_videos = youtube_videos.replace([np.inf, -np.inf], 0) ## inf

In [13]:
youtube_videos.to_csv('C:/py_src/awake/data/youtube_videos_credit.csv', encoding='utf-8-sig', index=False)

In [4]:
# youtube_videos = dd.read_csv('C:/py_src/awake/data/youtube_videos_credit.csv')
# youtube_videos = pd.read_csv(file_path + 'youtube_videos_credit.csv')

### 데이터 분할
- 미래를 예측하기 위한 모델이므로 시간순으로 데이터 분할
- 시작날짜 : 2023-03-26
- 종료날짜 : 2024-05-06
- 전체 기간의 80% 날짜 : 2024-02-11

In [14]:
# 컬럼 정리
unique_col = youtube_videos.columns[:3]
remove_col = ['estimatedRevenue','estimatedAdRevenue','grossRevenue','net_subscribers_change','subscribersGained','subscribersLost','engage_rate','views']
x_col = youtube_videos.columns.drop(list(unique_col) + remove_col) ## y값 제거, y값 변수 관여 변수 제거

In [15]:
# 데이터 분할
train_data = youtube_videos[youtube_videos['end_date'] <= '2024-02-11']
test_data = youtube_videos[youtube_videos['end_date'] > '2024-02-11']

In [16]:
# 불균형 확인
print(train_data.shape)
print(test_data.shape)

(6538190, 82)
(2031321, 82)


### 모델 기법 적용

In [17]:
# 각 모델 정의
# rf_model = RandomForestRegressor(random_state=42)
# gbm_model = GradientBoostingRegressor(random_state=42)
# lgbm_model = LGBMRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(random_state=42)

# 모델 리스트
models = {
    # 'RandomForest': rf_model,
    # 'GBM': gbm_model,
    # 'LightGBM': lgbm_model,
    'XGBoost': xgb_model
}

In [ ]:
# # 모델별 교차 검증 결과 저장
# results = {}

# for model_name, model in models.items():
#     print(f"\n{model_name} 모델 성능 평가 중...")
    
#     # 교차 검증
#     cv_scores = cross_val_score(model, train_data[x_col], train_data['subscribers_count'], cv=5, scoring='neg_mean_squared_error')
    
#     # 평균 RMSE 계산
#     rmse_scores = np.sqrt(-cv_scores)  # neg_mean_squared_error는 음수이므로 양수로 변환 후 제곱근
#     mean_rmse = rmse_scores.mean()
#     print(f"{model_name} 교차 검증 평균 RMSE: {mean_rmse}")
    
#     # 결과 저장
#     results[model_name] = mean_rmse

### 모델 성능 평가

In [260]:
feature_importances_dict = {}

In [261]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

for target in ['estimatedRevenue','net_subscribers_change','engage_rate']:
    # 모델 정의 및 학습
    models['XGBoost'].fit(train_data[x_col], train_data[target]) ## 'estimatedRevenue','net_subscribers_change','engage_rate

    # 변수 중요도
    feature_importances_dict[target] = models['XGBoost'].feature_importances_

    # 예측
    y_pred = models['XGBoost'].predict(test_data[x_col])

    # MSE 계산
    mse = mean_squared_error(test_data[target], y_pred)

    # RMSE 계산
    rmse = np.sqrt(mse)

    # R² 값 계산
    r2 = r2_score(test_data[target], y_pred)

    # Adjusted R² 계산
    n = len(test_data)  # 샘플 수
    p = test_data.shape[1]  # 독립 변수(특성) 수
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

    # 결과 출력
    print(target)
    print(f"R² 값: {r2:.4f}")
    print(f"Adjusted R² 값: {adjusted_r2:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print("----------------------------------------")
    print("")

estimatedRevenue
R² 값: 0.8992
Adjusted R² 값: 0.8992
MSE: 2389651.9695
RMSE: 1545.8499
----------------------------------------

net_subscribers_change
R² 값: 0.7636
Adjusted R² 값: 0.7636
MSE: 161.2195
RMSE: 12.6972
----------------------------------------

engage_rate
R² 값: 0.8314
Adjusted R² 값: 0.8314
MSE: 0.0013
RMSE: 0.0365
----------------------------------------



### 변수 중요도 확인
- 각 y값에 대한 중요도 평균

In [264]:
importances_df = pd.DataFrame({
    'features': x_col,
    'estimatedRevenue_importance': feature_importances_dict['estimatedRevenue'],
    'net_subscribers_change_importance': feature_importances_dict['net_subscribers_change'],
    'engage_rate_importance': feature_importances_dict['engage_rate']
    })

importances_df['mean_importance'] = importances_df[['estimatedRevenue_importance', 'net_subscribers_change_importance', 'engage_rate_importance']].mean(axis=1)

최종 지표 선정

In [265]:
# 신용평가 최종 지표 선정
final_col = list(importances_df.sort_values(['mean_importance'],ascending=False).iloc[:20]['features'])
final_col

['dislikes',
 'estimatedRedPartnerRevenue',
 'likes',
 'redViews',
 'monthly_revenue_diversification_ratio_trd',
 'shares',
 'revenue_diversification_ratio',
 'estimatedMinutesWatched',
 'revenue_per_view',
 'comments',
 'averageViewDuration',
 'playbackBasedCpm',
 'averageViewPercentage',
 'monthly_watch_time_per_subscriber_std',
 'watch_time_per_subscriber',
 'adImpressions',
 'monetizedPlaybacks',
 'monthly_estimatedRedPartnerRevenue_std',
 'subscriber_gain_per_content',
 'videosRemovedFromPlaylists']

In [ ]:
# # 신용평가 최종 지표
# final_col = ['dislikes', 'estimatedRedPartnerRevenue', 'likes', 'redViews', 'monthly_revenue_diversification_ratio_trd', 'shares', 'revenue_diversification_ratio', 'estimatedMinutesWatched',
#              'revenue_per_view', 'comments', 'averageViewDuration', 'playbackBasedCpm', 'averageViewPercentage', 'monthly_watch_time_per_subscriber_std', 'watch_time_per_subscriber',
#              'adImpressions', 'monetizedPlaybacks', 'monthly_estimatedRedPartnerRevenue_std', 'subscriber_gain_per_content', 'videosRemovedFromPlaylists']

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

for target in ['estimatedRevenue','net_subscribers_change','engage_rate']:
    # 모델 정의 및 학습
    models['XGBoost'].fit(train_data[final_col], train_data[target]) ## 'estimatedRevenue','net_subscribers_change','engage_rate

    # 변수 중요도
    feature_importances_dict[target] = models['XGBoost'].feature_importances_

    # 예측
    y_pred = models['XGBoost'].predict(test_data[final_col])

    # MSE 계산
    mse = mean_squared_error(test_data[target], y_pred)

    # RMSE 계산
    rmse = np.sqrt(mse)

    # R² 값 계산
    r2 = r2_score(test_data[target], y_pred)

    # Adjusted R² 계산
    n = len(test_data)  # 샘플 수
    p = test_data.shape[1]  # 독립 변수(특성) 수
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

    # 결과 출력
    print(target)
    print(f"R² 값: {r2:.4f}")
    print(f"Adjusted R² 값: {adjusted_r2:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print("----------------------------------------")
    print("")

estimatedRevenue
R² 값: 0.9054
Adjusted R² 값: 0.9054
MSE: 2242811.2569
RMSE: 1497.6018
----------------------------------------

net_subscribers_change
R² 값: 0.8259
Adjusted R² 값: 0.8259
MSE: 118.7260
RMSE: 10.8961
----------------------------------------

engage_rate
R² 값: 0.8329
Adjusted R² 값: 0.8329
MSE: 0.0013
RMSE: 0.0363
----------------------------------------



In [393]:
# import shap

# # SHAP Explainer 생성
# explainer = shap.TreeExplainer(model['XGBoost'])

# # SHAP 값 계산
# shap_values = explainer.shap_values(test_data[final_col])

# # SHAP 시각화
# shap.summary_plot(shap_values, test_data[final_col])


ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.1.

### 가중치 설정
- 최종 지표 활용

In [266]:
importances_df_final = importances_df[importances_df['features'].isin(final_col)][['features','mean_importance']].reset_index(drop=True).sort_values('mean_importance', ascending=False)
importances_df_final['weight'] = importances_df_final['mean_importance'] / importances_df_final['mean_importance'].sum() ## 가중치 설정
importances_df_final

,features,mean_importance,weight
3,dislikes,0.138300,0.159780
9,estimatedRedPartnerRevenue,0.131571,0.152006
2,likes,0.084882,0.098065
0,redViews,0.052842,0.061049
19,monthly_revenue_diversification_ratio_trd,0.042292,0.048860
4,shares,0.038931,0.044977
14,revenue_diversification_ratio,0.038377,0.044338
5,estimatedMinutesWatched,0.036728,0.042432
13,revenue_per_view,0.036697,0.042397
1,comments,0.035313,0.040798


평가요소 분류

In [284]:
# 상환이력
eval_col1 = ['dislikes','likes','shares','comments','redViews']
importances_df_final[importances_df_final['features'].isin(eval_col1)]['weight'].sum()

np.float32(0.4046696)

In [285]:
# 부채수준
eval_col2 = ['estimatedRedPartnerRevenue','monthly_estimatedRedPartnerRevenue_std','revenue_per_view']
importances_df_final[importances_df_final['features'].isin(eval_col2)]['weight'].sum()

np.float32(0.2153935)

In [286]:
# 신용거래기간
eval_col3 = ['subscriber_gain_per_content','videosRemovedFromPlaylists','monthly_watch_time_per_subscriber_std','watch_time_per_subscriber']
importances_df_final[importances_df_final['features'].isin(eval_col3)]['weight'].sum()

np.float32(0.09121278)

In [287]:
# 신용형태
eval_col4 = ['revenue_diversification_ratio','monthly_revenue_diversification_ratio_trd','playbackBasedCpm','monetizedPlaybacks','adImpressions']
importances_df_final[importances_df_final['features'].isin(eval_col4)]['weight'].sum()

np.float32(0.17576551)

In [288]:
# 비금융/마이데이터
eval_col5 = ['averageViewDuration','averageViewPercentage','estimatedMinutesWatched']
importances_df_final[importances_df_final['features'].isin(eval_col5)]['weight'].sum()

np.float32(0.11295858)

가중치 적용

In [289]:
# 가중치 실제값 적용
# 상환이력
youtube_videos['score1'] = (youtube_videos[eval_col1] * np.array(importances_df_final[importances_df_final['features'].isin(eval_col1)]['weight'])).sum(axis=1)

In [290]:
# 부채수준
youtube_videos['score2'] = (youtube_videos[eval_col2] * np.array(importances_df_final[importances_df_final['features'].isin(eval_col2)]['weight'])).sum(axis=1)

In [291]:
# 신용거래기간
youtube_videos['score3'] = (youtube_videos[eval_col3] * np.array(importances_df_final[importances_df_final['features'].isin(eval_col3)]['weight'])).sum(axis=1)

In [292]:
# 신용형태
youtube_videos['score4'] = (youtube_videos[eval_col4] * np.array(importances_df_final[importances_df_final['features'].isin(eval_col4)]['weight'])).sum(axis=1)

In [293]:
# 비금융/마이데이터
youtube_videos['score5'] = (youtube_videos[eval_col5] * np.array(importances_df_final[importances_df_final['features'].isin(eval_col5)]['weight'])).sum(axis=1)

In [33]:
# # 평가요소 분류 없이 전체 합
# youtube_videos['score'] = (youtube_videos[importances_df_final['features']] * np.array(importances_df_final['weight'])).sum(axis=1)

In [294]:
# 계정별 신용점수 확인 - 계정별 평가요소 합계
credit_df = youtube_videos.groupby('youtube_user_id')[['score1','score2','score3','score4','score5']].sum().reset_index()

In [295]:
# 불필요 및 이상치 제거
credit_df = credit_df[~credit_df['youtube_user_id'].isin(['639bb8dcd603b8138e33780b'])].reset_index(drop=True)
# '639bb8dcd603b8138e33780b' ## 없는 계정 및 이상치

In [681]:
from sklearn.preprocessing import MinMaxScaler

# 1. MinMaxScaler 적용
scaler = MinMaxScaler()
# credit_df['score_scale'] = scaler.fit_transform(credit_df[['score']])
credit_df['score1_scale'] = scaler.fit_transform(credit_df[['score1']])
credit_df['score2_scale'] = scaler.fit_transform(credit_df[['score2']])
credit_df['score3_scale'] = scaler.fit_transform(credit_df[['score3']])
credit_df['score4_scale'] = scaler.fit_transform(credit_df[['score4']])
credit_df['score5_scale'] = scaler.fit_transform(credit_df[['score5']])

# credit_df['score_final'] = scaler.fit_transform(credit_df[['score_scale']]) * 1000
# credit_df['score1_final'] = scaler.fit_transform(credit_df[['score1_scale']]) * 350 # 평가요소 0.284 * 0 이 아닌 비율 0.12 --> 0.03 : 0.075 * 0.35 --> 0.03  : 0.15
# credit_df['score2_final'] = scaler.fit_transform(credit_df[['score2_scale']]) * 280 # 평가요소 0.245 * 0 이 아닌 비율 0.46 --> 0.11 : 0.27 * 0.28 --> 0.076 : 0.38
# credit_df['score3_final'] = scaler.fit_transform(credit_df[['score3_scale']]) * 120 # 평가요소 0.123 * 0 이 아닌 비율 0.25 --> 0.03 : 0.075 * 0.12 --> 0.009 : 0.04
# credit_df['score4_final'] = scaler.fit_transform(credit_df[['score4_scale']]) * 180 # 평가요소 0.275 * 0 이 아닌 비율 0.58 --> 0.16 : 0.4 * 0.18 --> 0.072 : 0.36
# credit_df['score5_final'] = scaler.fit_transform(credit_df[['score5_scale']]) * 80 # 평가요소 0.073 * 0 이 아닌 비율 0.9 --> 0.07 : 0.18 * 0.08 --> 0.014 : 0.07

credit_df['score1_final'] = scaler.fit_transform(credit_df[['score1_scale']]) * 430
credit_df['score2_final'] = scaler.fit_transform(credit_df[['score2_scale']]) * 410
credit_df['score3_final'] = scaler.fit_transform(credit_df[['score3_scale']]) * 50
credit_df['score4_final'] = scaler.fit_transform(credit_df[['score4_scale']]) * 60
credit_df['score5_final'] = scaler.fit_transform(credit_df[['score5_scale']]) * 50

In [682]:
# 평가요소 분류 활용 스코어링
credit_df['credit_score'] = credit_df[['score1_final','score2_final','score3_final','score4_final','score5_final']].sum(axis=1)

In [683]:
# 계정 확인 테이블
user_info_df = merge_df_users_fin[['youtube_user_id','channel_title']].drop_duplicates().reset_index(drop=True)
user_info_df = user_info_df[user_info_df['channel_title']!='0'].drop_duplicates().reset_index(drop=True)
user_info_df = user_info_df.drop_duplicates('youtube_user_id').reset_index(drop=True)
user_info_df = user_info_df.drop_duplicates('channel_title').reset_index(drop=True)

In [684]:
# 신용평가점수 테이블
credit_df_fin = pd.merge(credit_df[['youtube_user_id','credit_score']],user_info_df,how='left',on='youtube_user_id')
credit_df_fin = credit_df_fin[['youtube_user_id','channel_title','credit_score']]
credit_df_fin = credit_df_fin[~credit_df_fin['channel_title'].isnull()].reset_index(drop=True)

In [688]:
credit_df_fin.sort_values(['credit_score'],ascending=False)

,youtube_user_id,channel_title,credit_score
90,63d77c9650eb530dfd139f8b,kiu기우쌤,811.211271
98,63eb4f87ee122e631992279f,수빙수tv sooBingsoo,607.087625
128,6401e117d746c60e1271fdef,앙찡,501.772071
31,629f6ca6eaf5732d6df0611e,Mind Patting마음토닥,487.574932
199,64da8a9ef638790e0f74bae7,잼스기타,394.203490
...,...,...,...
48,62bc1aca507271632b940e2e,도아이 Doh-I,0.049373
20,6287a9cefb15712a8cbb098e,혜성네일_comet,0.044330
68,6314a287a0673403176d3c35,gahyun 가현,0.024643
57,62d11f9f0b4c4c7502a5c1b6,DDONIE 또니 / 러브크레센트,0.019391


### 변동계수
- y값 활용 : 'estimatedRevenue', 'net_subscribers_change', 'engage_rate'
- 0값이 많아 왜곡된 변동계수를 계산할 수 있음 --> 0값 제거후 변동계수 산출

In [689]:
# estimatedRevenue
youtube_videos['weekly_estimatedRevenue_cv'] = youtube_videos.groupby('video')['estimatedRevenue'].transform(lambda x: x.rolling(window=7).std() / x.rolling(window=7).mean())
youtube_videos['monthly_estimatedRevenue_cv'] = youtube_videos.groupby('video')['estimatedRevenue'].transform(lambda x: x.rolling(window=30).std() / x.rolling(window=30).mean())

# net_subscribers_change
youtube_videos['weekly_net_subscribers_change_cv'] = youtube_videos[youtube_videos['net_subscribers_change']!=0].groupby('video')['net_subscribers_change'].transform(lambda x: x.rolling(window=7).std() / x.rolling(window=7).mean())
youtube_videos['monthly_net_subscribers_change_cv'] = youtube_videos[youtube_videos['net_subscribers_change']!=0].groupby('video')['net_subscribers_change'].transform(lambda x: x.rolling(window=30).std() / x.rolling(window=30).mean())

# engage_rate
youtube_videos['weekly_engage_rate_cv'] = youtube_videos[youtube_videos['engage_rate']!=0].groupby('video')['engage_rate'].transform(lambda x: x.rolling(window=7).std() / x.rolling(window=7).mean())
youtube_videos['monthly_engage_rate_cv'] = youtube_videos[youtube_videos['engage_rate']!=0].groupby('video')['engage_rate'].transform(lambda x: x.rolling(window=30).std() / x.rolling(window=30).mean())

cv_col = ['weekly_estimatedRevenue_cv','monthly_estimatedRevenue_cv','weekly_net_subscribers_change_cv','monthly_net_subscribers_change_cv','weekly_engage_rate_cv','monthly_engage_rate_cv']

In [690]:
# null값 대체
youtube_videos = youtube_videos.fillna(0) ## NaN
youtube_videos = youtube_videos.replace([np.inf, -np.inf], 0) ## inf

In [691]:
# 계정별 변동계수 평균
coefvar_df = youtube_videos.groupby('youtube_user_id')[cv_col].mean().reset_index()

In [694]:
# 최종 신용평가 테이블
credit_coef_df = pd.merge(credit_df_fin, coefvar_df, how='left', on='youtube_user_id')
# credit_df_final = credit_df_final[~credit_df_final['channel_title'].isnull()].reset_index(drop=True)

In [718]:
youtube_videos[youtube_videos['youtube_user_id']=='6314a287a0673403176d3c35']

,youtube_user_id,video,end_date,views,redViews,comments,likes,dislikes,shares,estimatedMinutesWatched,...,score2,score3,score4,score5,weekly_estimatedRevenue_cv,monthly_estimatedRevenue_cv,weekly_net_subscribers_change_cv,monthly_net_subscribers_change_cv,weekly_engage_rate_cv,monthly_engage_rate_cv
11829,6314a287a0673403176d3c35,-5PwDakTEzU,2024-03-03,109,36,2,3,0,1,155,...,0.0,0.085457,0.0,9.405768,0.0,0.0,0.0,0.0,0.0,0.0
11830,6314a287a0673403176d3c35,-5PwDakTEzU,2024-03-04,56,22,0,0,0,0,49,...,0.0,0.000000,0.0,4.245584,0.0,0.0,0.0,0.0,0.0,0.0
11831,6314a287a0673403176d3c35,-5PwDakTEzU,2024-03-05,49,19,1,1,0,0,64,...,0.0,0.028486,0.0,6.094789,0.0,0.0,0.0,0.0,0.0,0.0
11832,6314a287a0673403176d3c35,-5PwDakTEzU,2024-03-06,22,6,0,1,0,0,28,...,0.0,0.000000,0.0,4.857262,0.0,0.0,0.0,0.0,0.0,0.0
11833,6314a287a0673403176d3c35,-5PwDakTEzU,2024-03-07,2,0,0,0,0,0,9,...,0.0,0.000000,0.0,14.673851,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462863,6314a287a0673403176d3c35,zBllPPtEdgE,2023-10-20,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8462864,6314a287a0673403176d3c35,zBllPPtEdgE,2023-10-29,0,0,0,0,0,0,0,...,0.0,0.028486,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8462865,6314a287a0673403176d3c35,zBllPPtEdgE,2023-11-13,0,0,0,0,0,0,0,...,0.0,0.028486,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8462866,6314a287a0673403176d3c35,zBllPPtEdgE,2024-02-16,0,0,0,0,0,0,0,...,0.0,0.028486,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [695]:
credit_coef_df.sort_values('credit_score',ascending=False)

,youtube_user_id,channel_title,credit_score,weekly_estimatedRevenue_cv,monthly_estimatedRevenue_cv,weekly_net_subscribers_change_cv,monthly_net_subscribers_change_cv,weekly_engage_rate_cv,monthly_engage_rate_cv
90,63d77c9650eb530dfd139f8b,kiu기우쌤,811.211271,0.279358,0.390812,0.173795,0.164415,0.336538,0.311575
98,63eb4f87ee122e631992279f,수빙수tv sooBingsoo,607.087625,0.258054,0.315873,0.146630,0.135114,0.296946,0.281829
128,6401e117d746c60e1271fdef,앙찡,501.772071,0.295230,0.459919,0.189307,0.202878,0.274237,0.261040
31,629f6ca6eaf5732d6df0611e,Mind Patting마음토닥,487.574932,1.043425,0.946792,0.038279,0.024342,0.059722,0.023763
199,64da8a9ef638790e0f74bae7,잼스기타,394.203490,0.312926,0.446608,0.189778,0.166439,0.283719,0.240656
...,...,...,...,...,...,...,...,...,...
48,62bc1aca507271632b940e2e,도아이 Doh-I,0.049373,0.000000,0.000000,0.000275,0.000000,0.054838,0.052217
20,6287a9cefb15712a8cbb098e,혜성네일_comet,0.044330,0.722904,0.766164,0.000000,0.000000,0.001901,0.000000
68,6314a287a0673403176d3c35,gahyun 가현,0.024643,0.000000,0.000000,0.000000,0.000000,0.001904,0.000000
57,62d11f9f0b4c4c7502a5c1b6,DDONIE 또니 / 러브크레센트,0.019391,0.000000,0.000000,0.000000,0.000000,0.008659,0.000000


In [710]:
cv_col = ['weekly_estimatedRevenue_cv', 'monthly_estimatedRevenue_cv', 'weekly_net_subscribers_change_cv', 'monthly_net_subscribers_change_cv', 'weekly_engage_rate_cv', 'monthly_engage_rate_cv']
credit_coef_df['CV_score'] = credit_coef_df[cv_col].mean(axis=1)

In [711]:
credit_coef_df

,youtube_user_id,channel_title,credit_score,weekly_estimatedRevenue_cv,monthly_estimatedRevenue_cv,weekly_net_subscribers_change_cv,monthly_net_subscribers_change_cv,weekly_engage_rate_cv,monthly_engage_rate_cv,weekly_estimatedRevenue_cv_score,monthly_estimatedRevenue_cv_score,weekly_net_subscribers_change_cv_score,monthly_net_subscribers_change_cv_score,weekly_engage_rate_cv_score,monthly_engage_rate_cv_score,CV_score
0,627cb611aa6f212355e0b617,성팩 SPAAK,13.349272,0.490795,0.574736,0.040273,0.035230,0.117803,0.100668,45.828482,38.273774,71.979530,72.357771,74.986730,76.443230,0.226584
1,627f59ccaa39226247c60b01,고도람 Go!doram,0.476567,0.802761,0.809237,0.000000,0.000000,0.009414,0.001670,17.751511,17.168671,75.000000,75.000000,84.199827,84.858027,0.270514
2,6287228afb15712a8cb931d7,세남자 물고기,0.524080,0.240621,0.369765,0.012032,0.010465,0.075025,0.062087,68.344083,56.721108,74.097620,74.215107,78.622900,79.722614,0.128333
3,6287229efb15712a8cb93225,띠혜 ddihye,79.980751,0.298516,0.386913,0.036342,0.019539,0.188688,0.149371,63.133555,55.177824,72.274322,73.534600,68.961536,72.303430,0.179895
4,628722c8fb15712a8cb9326e,소리미의 신화방송,5.023647,1.286501,1.491383,0.001229,0.000380,0.032041,0.015675,-25.785126,-44.224480,74.907790,74.971502,82.276536,83.667630,0.471202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,65cc401305bf1c0baa425146,주피코,144.009275,0.344102,0.314013,0.294535,0.176194,0.227461,0.146422,59.030789,61.738801,52.909880,61.785421,65.665793,72.554091,0.250455
230,65e7b773d8da110bb072e2b5,신크TV,1.788091,0.783319,0.254329,0.000160,0.000000,0.001754,0.000000,19.501308,67.110398,74.987990,75.000000,84.850896,85.000000,0.173260
231,65f7b17ed8da110bb0733b7b,Yerendipity예렌디피티,0.227405,0.276907,0.429779,0.018331,0.002414,0.035343,0.003394,65.078353,51.319872,73.625212,74.818983,81.995831,84.711509,0.127695
232,65fecf7ed8da110bb0736199,JN테크리뷰,6.575062,0.595728,0.234931,0.016795,0.000595,0.072448,0.008624,36.384517,68.856212,73.740353,74.955385,78.841933,84.266921,0.154854


In [701]:
# 변동계수 가중치 적용
credit_coef_df['weekly_estimatedRevenue_cv_score'] = (1 - credit_coef_df['weekly_estimatedRevenue_cv']) * 90
credit_coef_df['monthly_estimatedRevenue_cv_score'] = (1 - credit_coef_df['monthly_estimatedRevenue_cv']) * 90
credit_coef_df['weekly_net_subscribers_change_cv_score'] = (1 - credit_coef_df['weekly_net_subscribers_change_cv']) * 75
credit_coef_df['monthly_net_subscribers_change_cv_score'] = (1 - credit_coef_df['monthly_net_subscribers_change_cv']) * 75
credit_coef_df['weekly_engage_rate_cv_score'] = (1 - credit_coef_df['weekly_engage_rate_cv']) * 85
credit_coef_df['monthly_engage_rate_cv_score'] = (1 - credit_coef_df['monthly_engage_rate_cv']) * 85

credit_coef_df['CV_score'] = credit_coef_df[['weekly_estimatedRevenue_cv_score','monthly_estimatedRevenue_cv_score','weekly_net_subscribers_change_cv_score','monthly_net_subscribers_change_cv_score','weekly_engage_rate_cv_score','monthly_engage_rate_cv_score']].sum(axis=1)

In [712]:
# 최종 데이터셋 확인
credit_df_final = credit_coef_df[['youtube_user_id','channel_title','credit_score','CV_score']]
credit_df_final['SCORE'] = credit_df_final['credit_score'] + credit_df_final['CV_score'] ## 최종 스코어 산출

C:\Users\kkims\AppData\Local\Temp\ipykernel_36860\2670103025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit_df_final['SCORE'] = credit_df_final['credit_score'] + credit_df_final['CV_score'] ## 최종 스코어 산출


In [717]:
credit_coef_df[['youtube_user_id','channel_title','credit_score'] + cv_col].sort_values('credit_score',ascending=False)

,youtube_user_id,channel_title,credit_score,weekly_estimatedRevenue_cv,monthly_estimatedRevenue_cv,weekly_net_subscribers_change_cv,monthly_net_subscribers_change_cv,weekly_engage_rate_cv,monthly_engage_rate_cv
90,63d77c9650eb530dfd139f8b,kiu기우쌤,811.211271,0.279358,0.390812,0.173795,0.164415,0.336538,0.311575
98,63eb4f87ee122e631992279f,수빙수tv sooBingsoo,607.087625,0.258054,0.315873,0.146630,0.135114,0.296946,0.281829
128,6401e117d746c60e1271fdef,앙찡,501.772071,0.295230,0.459919,0.189307,0.202878,0.274237,0.261040
31,629f6ca6eaf5732d6df0611e,Mind Patting마음토닥,487.574932,1.043425,0.946792,0.038279,0.024342,0.059722,0.023763
199,64da8a9ef638790e0f74bae7,잼스기타,394.203490,0.312926,0.446608,0.189778,0.166439,0.283719,0.240656
...,...,...,...,...,...,...,...,...,...
48,62bc1aca507271632b940e2e,도아이 Doh-I,0.049373,0.000000,0.000000,0.000275,0.000000,0.054838,0.052217
20,6287a9cefb15712a8cbb098e,혜성네일_comet,0.044330,0.722904,0.766164,0.000000,0.000000,0.001901,0.000000
68,6314a287a0673403176d3c35,gahyun 가현,0.024643,0.000000,0.000000,0.000000,0.000000,0.001904,0.000000
57,62d11f9f0b4c4c7502a5c1b6,DDONIE 또니 / 러브크레센트,0.019391,0.000000,0.000000,0.000000,0.000000,0.008659,0.000000


### 실제데이터 결과 확인

In [14]:
# 예측 결과 확인
# y_pred = stacking_model.predict(merge_df_users_fin[x_col])
# merge_df_users_fin['predict'] = y_pred

y_pred = models['XGBoost'].predict(youtube_videos[x_col])
youtube_videos['predict'] = y_pred

In [15]:
# 계정별 구독자수 평균, 구독자 예측수 평균 비교
result_contents_df = youtube_videos[['youtube_user_id', 'video', 'end_date', 'estimatedAdRevenue', 'predict']]

# 계정별 콘텐츠의 구독자 순증감 1일 합계
result_contents_df = result_contents_df.groupby(['youtube_user_id', 'end_date'], as_index=False).agg({
    'estimatedAdRevenue': 'sum',
    'predict': 'sum'
})

# 1개월, 3개월, 6개월, 1년 뒤 구독자수 예측값 계산
result_contents_df['end_date'] = pd.to_datetime(result_contents_df['end_date'])

# Shift와 Rolling 연산을 위한 그룹별 처리
result_contents_df['1_month_future_predict'] = result_contents_df.groupby('youtube_user_id')['predict'].transform(lambda x: x.shift(-30).rolling(window=30).sum())
result_contents_df['3_month_future_predict'] = result_contents_df.groupby('youtube_user_id')['predict'].transform(lambda x: x.shift(-90).rolling(window=90).sum())
result_contents_df['6_month_future_predict'] = result_contents_df.groupby('youtube_user_id')['predict'].transform(lambda x: x.shift(-180).rolling(window=180).sum())
result_contents_df['12_month_future_predict'] = result_contents_df.groupby('youtube_user_id')['predict'].transform(lambda x: x.shift(-365).rolling(window=365).sum())

# 계정별로 최종 평균값을 계산
result_contents_df_final = result_contents_df.groupby('youtube_user_id').agg({
    'estimatedAdRevenue': 'mean',
    'predict': 'mean',
    '1_month_future_predict': 'mean',
    '3_month_future_predict': 'mean',
    '6_month_future_predict': 'mean',
    '12_month_future_predict': 'mean'
}).reset_index()


In [21]:
result_contents_df_final[result_contents_df_final['predict'] > 0]

,youtube_user_id,estimatedAdRevenue,predict,1_month_future_predict,3_month_future_predict,6_month_future_predict,12_month_future_predict
0,627cb611aa6f212355e0b617,7776.284805,7865.420898,238511.455694,615687.204672,1.276685e+06,NaN
1,627f59ccaa39226247c60b01,22.858974,28.559135,857.206126,2612.180373,4.973132e+03,NaN
2,6287228afb15712a8cb931d7,118.569240,119.170174,2175.118191,243.520620,3.429281e+02,NaN
3,6287229efb15712a8cb93225,319.104255,412.594482,10763.081399,22723.167320,6.502604e+04,NaN
4,628722c8fb15712a8cb9326e,584.817873,616.643494,17610.511524,48419.635411,8.952879e+04,NaN
...,...,...,...,...,...,...,...
244,65cc401305bf1c0baa425146,1177.050562,1004.203003,31746.118715,NaN,NaN,NaN
245,65e7b773d8da110bb072e2b5,2419.026120,2434.887939,80514.173564,NaN,NaN,NaN
246,65f7b17ed8da110bb0733b7b,150.755880,154.831406,NaN,NaN,NaN,NaN
247,65fecf7ed8da110bb0736199,14030.064715,14011.172852,NaN,NaN,NaN,NaN


In [33]:
top_user_id_20_6_month = result_contents_df_final[~result_contents_df_final['6_month_future_predict'].isnull()].sort_values(['6_month_future_predict']).iloc[-20:]['youtube_user_id']
bottom_user_id_20_6_month = result_contents_df_final[~result_contents_df_final['6_month_future_predict'].isnull()].sort_values(['6_month_future_predict']).iloc[:20]['youtube_user_id']

In [44]:
merge_df_users_fin[merge_df_users_fin['youtube_user_id'].isin(top_user_id_20_6_month)]['channel_title'].unique()

array(['Mind Patting마음토닥', '0', '모염 moyeom', 'abbapraise 아바프레이즈', '유경몬',
       '콜드쉽 Coldsheep', '월텍남 - 월스트리트 테크남', '팀브라더스', 'kiu기우쌤', '비됴클래스',
       '수빙수tv sooBingsoo', '만능혁키', '석시원 커플 SeokSiWon Couple', '너굴몬',
       '코인덕 차트아지', '미니멀영어 Minimal English', '윤순의 평범치 않은 생활',
       '日本ジヌ【니혼지누】ー韓国に関する全て', '뻘짓연구소', '중년독수리의 대리여행', '북토크'], dtype=object)

In [45]:
merge_df_users_fin[merge_df_users_fin['youtube_user_id'].isin(bottom_user_id_20_6_month)]['channel_title'].unique()

array(['세남자 물고기', '0', '름쿠 ᴘʟᴀʏʟɪꜱᴛ', '마파TV', 'the sence', '다먹어라이언',
       'sa lly', '도아이 Doh-I', 'ORlGN 오리진', '윈플즈TV', 'DDONIE 또니 / 러브크레센트',
       'gahyun 가현', '인썸니아TV', '한나임한나Hannaim', '로컬필름 LOCAL FILM', '꾸앤끄',
       '나나무비', '키키낙낙', '탬니몰리', 'WORKS.D PLAYLIST'], dtype=object)

In [73]:
result_contents_df_final[result_contents_df_final['estimatedAdRevenue']!=0].sort_values(['predict'])

,youtube_user_id,estimatedAdRevenue,predict,1_month_future_predict,3_month_future_predict,6_month_future_predict,12_month_future_predict
21,6287a9cefb15712a8cbb098e,7.386412,8.279515,2.809071e+02,NaN,NaN,NaN
28,629b694beaf5732d6deae948,11.077663,13.263415,3.710075e+02,1.012134e+03,NaN,NaN
204,64c7fc2a1951980e344809f0,11.443656,13.787192,6.959918e+01,1.633820e+02,NaN,NaN
79,6353096a5a3ac10b5fe8376b,17.244489,25.759954,7.142252e+02,2.043089e+03,3.939559e+03,NaN
47,62b3c78d507271632b8ade02,11.523946,28.041706,8.220679e+02,2.572409e+03,5.192340e+03,NaN
...,...,...,...,...,...,...,...
173,645ec17eef566f0e136a9880,152430.860845,152357.812500,4.481848e+06,1.380492e+07,2.768449e+07,NaN
57,62d11f080b4c4c7502a5be3d,311574.550930,311694.593750,9.304204e+06,2.896338e+07,5.822894e+07,NaN
108,63eb4f87ee122e631992279f,323549.499186,324879.625000,9.218749e+06,2.773670e+07,4.785841e+07,NaN
84,639bb8dcd603b8138e33780b,367042.582846,354395.218750,1.178914e+07,4.438877e+07,NaN,NaN


In [69]:
merge_df_users_fin[merge_df_users_fin['youtube_user_id'] == '62b3c78d507271632b8ade02']

,youtube_user_id,date,channel_id,channel_title,phone_num,report_user_id,published_at,viewCount,subscriberCount,videoCount,...,weekly_estimated_revenue,weekly_total_view_time,adult_viewer_rate,quarterly_subscribers_lost,weekly_subscribers_lost,red_revenue_rate,monthly_subscribers_lost,estimatedMinutesWatched,monthly_estimated_revenue,estimated_ad_revenue
19500,62b3c78d507271632b8ade02,2023-03-26,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,12343.587,15887.0,89.0,670.0,38.0,0.000675,198.0,7018.0,45030.278,15.456
19501,62b3c78d507271632b8ade02,2023-03-27,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,13218.991,20900.0,82.4,669.0,33.0,0.000542,194.0,6728.0,46175.592,39.989
19502,62b3c78d507271632b8ade02,2023-03-28,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,15056.954,28481.0,87.7,671.0,33.0,0.000726,191.0,9125.0,47926.061,10.500
19503,62b3c78d507271632b8ade02,2023-03-29,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,14141.809,34739.0,87.8,663.0,31.0,0.000144,187.0,8050.0,48978.677,32.627
19504,62b3c78d507271632b8ade02,2023-03-30,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,16089.837,43657.0,86.8,658.0,31.0,0.000483,187.0,10313.0,50708.798,25.967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19900,62b3c78d507271632b8ade02,2024-04-29,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,5087.921,8462.0,92.7,617.0,22.0,0.216267,143.0,1098.0,24905.241,17.482
19901,62b3c78d507271632b8ade02,2024-04-30,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,5167.600,8697.0,90.1,616.0,24.0,0.169105,143.0,1379.0,24246.425,0.000
19902,62b3c78d507271632b8ade02,2024-05-01,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,5266.899,8908.0,89.1,608.0,25.0,0.187596,140.0,1232.0,24194.967,5.887
19903,62b3c78d507271632b8ade02,2024-05-02,UC67YKqgxk9eGSVBz2mt35aw,수집의 수집,010-9641-1969,0,2011-02-16 19:31:52.000,11620880.0,14400.0,553.0,...,5279.221,9066.0,90.3,606.0,25.0,0.188938,142.0,1271.0,24210.491,6.763
